# --- Day 15: Beacon Exclusion Zone ---



In [1]:
def calc_dist(pos_a, pos_b):
    # Calculate Manhattan distance
    return abs(pos_b[0] - pos_a[0]) + abs(pos_b[1] - pos_a[1])

def read_input(filename):
    # Parse into list
    # Each element of list corresponds to a sensor
    # Each element is a triple [sensor, beacon, distance]
    input_data = list()
    with open(filename) as infile:
        for line in infile:
            sensor, beacon = line.strip().replace("Sensor at ", "").split(": closest beacon is at ")
            sensor_row = [tuple([int(x.split("=")[1]) for x in y.split(", ")]) for y in [sensor, beacon]]
            sensor_row.append(calc_dist(sensor_row[1], sensor_row[0]))
            input_data.append(sensor_row)
                               
    return input_data


In [2]:
import numpy as np
from tqdm import tqdm


def pretty_print_matrix(mat):
    for i in range(0, len(mat)):
        print(
            "".join(
                [
                    "." if y == 0 else "S" if y == -1 else "B" if y == -2 else "#"
                    for y in mat[i]
                ]
            )
        )
    return 0

def create_matrix(sensor_data, row_number, row_width, part1=True):
    x_min = min(min(x[0][0] - x[2], x[1][0] - x[2]) for x in sensor_data)
    x_max = max(max(x[0][0] + x[2], x[1][0] + x[2]) for x in sensor_data)
    y_min = min(min(x[0][1] - x[2], x[1][1] - x[2]) for x in sensor_data)
    y_max = max(max(x[0][1] + x[2], x[1][1] + x[2]) for x in sensor_data) 
    print(f"x_min: {x_min}")
    print(f"x_max: {x_max}")
    print(f"y_min: {y_min}")
    print(f"y_max: {y_max}")
    if part1:
        m = np.full((1+(2*row_width), x_max-x_min), 0) 
    else:
        m = np.full((1+(2*row_width), x_max-x_min), 0, dtype='uint8')
    
    #[ ] [ ] [ ] [ ]
    #[ ] [ ] [ ] [ ]
    # block by block
    row_range = range((row_number-y_min)-row_width, (row_number-y_min)+row_width+1)
    row_offset = row_number - row_width
    #print(f"row_range: {list(row_range)}")
    #pretty_print_matrix(m)
    for r in sensor_data:
        #print(r[0])
        #print(r[1])
        #print(r[1][1]-y_min)
        if part1:
            if r[0][1]-y_min in row_range:
                print(f"sensor at: {r[0]} -> ({r[0][1]-row_offset}, {r[0][0]-x_min})")
                m[r[0][1]-row_offset, r[0][0]-x_min] = -1

            if r[1][1]-y_min in row_range:
                print(f"beacon at: {r[1]} -> ({r[1][1]-row_offset}, {r[1][0]-x_min})")
                m[r[1][1]-row_offset, r[1][0]-x_min] = -2
            
        indices = np.indices(m.shape)
        y_indices = indices[0]
        x_indices = indices[1]

        locations = abs(x_indices - (r[0][0]-x_min)) + abs(y_indices - (r[0][1]-row_offset)) <= r[2]
        m[locations & (m==0)]=1
    
    #pretty_print_matrix(m)
    
    print(f"Row y={row_number} has {sum(m[row_width, :]==1)} positions where a beacon cannot be present.")
        
        #pretty_print_matrix(m[range(row_number-1, row_number+2), :])
    
    return None


input_data = read_input("inputs/day15.txt")
create_matrix(input_data, 2000000, 1);


x_min: -1789570
x_max: 5315149
y_min: -2035378
y_max: 5428150
beacon at: (-85806, 2000000) -> (1, 1703764)
beacon at: (-85806, 2000000) -> (1, 1703764)
beacon at: (-85806, 2000000) -> (1, 1703764)
Row y=2000000 has 4811413 positions where a beacon cannot be present.


# Alternative approach 

In [54]:
def sensor_more_top_left(sensor1, sensor2):
    if (sensor1[0][0]+sensor1[0][1]) <= (sensor2[0][0]+sensor2[0][1]):
        return True
    else:
        return False

def sort_sensors(sensor_data_to_sort):
    i = 1
    while i < len(sensor_data_to_sort):
        j = i
        while (
            j > 0
            and sensor_more_top_left(sensor_data_to_sort[j], sensor_data_to_sort[j-1])
        ):
            sensor_data_to_sort[j - 1], sensor_data_to_sort[j] = sensor_data_to_sort[j], sensor_data_to_sort[j - 1]
            j -= 1

        i += 1

    return sensor_data_to_sort


def split_chunk(chunks, search_size):
    print(f"splitting {len(chunks)} chunks")
    split_chunks = list()
    for big_chunk in chunks:
        chunk_size = (big_chunk[1][0]-big_chunk[0][0])+1

        new_chunk_size = chunk_size//2
        
        if len(split_chunks) == 0:
            print(f"chunk_size: {chunk_size}")            
            print(f"new chunk size: {new_chunk_size}")
        
        #print(f"big_chunk: {big_chunk}")
        #print(f"big_chunk[0][0]: {big_chunk[0][0]}")
        new_chunks = list()
        for i in range(0,2):
            for j in range(0,2):
                #print(f"i={i},j={j}")
                x_min = big_chunk[0][0] + i*new_chunk_size
                x_max = big_chunk[0][0] + (i+1)*(new_chunk_size) - 1
                y_min = big_chunk[0][1] + j*new_chunk_size
                y_max = big_chunk[0][1] + (j+1)*(new_chunk_size) - 1
                chunk_corners = [(x_min, y_min), (x_max, y_min), (x_min, y_max), (x_max, y_max)]
                #print(chunk_corners)
                if (x_min <= search_size) or (y_min <= search_size):
                    new_chunks.append(chunk_corners)
        split_chunks += new_chunks
    split_chunks = list(map(list,set(map(tuple,split_chunks))))
    print(f"split into {len(split_chunks)} chunks.")

    return split_chunks
    
import time 
def is_chunk_covered(sensor_data, chunkle, search_size):
    
    for x in range(chunkle[0][0], chunkle[3][0]+1):
        for y in range(chunkle[0][1], chunkle[3][1]+1):
            #print((x,y))
            if all([calc_dist((x,y), sensor[0]) > sensor[2] for sensor in sensor_data]):
                if (x >= 0) and (x <= search_size) and (y >= 0) and (y <= search_size):
                    print(f"x={x}, y={y}")
                    return False 
    
    return True

xmin = 10
ymin = 10
size = 63
chunk = [(xmin, ymin), (xmin+size, ymin), (xmin, ymin+size), (xmin+size, ymin+size)]
#print(chunk)
split_chunk(split_chunk([chunk], 500), 500);
#is_chunk_covered(input_data, chunk, 20)


splitting 1 chunks
chunk_size: 64
new chunk size: 32
split into 4 chunks.
splitting 4 chunks
chunk_size: 32
new chunk size: 16
split into 16 chunks.


In [65]:
def part2(sensor_data, search_size, chunk_size, granularity_limit):
    # split search space into chunks
    sensor_data = sort_sensors(sensor_data)    
    
    # generate stack of chunks
    # chunk is defined by its corners
    chunks = []
    
    # add edge chunks
    search_size
    
    # number of chunks along an edge
    num_chunks_side = (search_size+chunk_size)//chunk_size
        
    print(f"number of chunks: {num_chunks_side}*{num_chunks_side} = {num_chunks_side*num_chunks_side}")
    # horizontal
    for i in range(0, num_chunks_side):
        # vertical
        for j in range(0, num_chunks_side):
            x_min = i*chunk_size
            x_max = (i+1)*(chunk_size) - 1
            y_min = j*chunk_size
            y_max = (j+1)*(chunk_size) - 1
            chunk_corners = [(x_min, y_min), (x_max, y_min), (x_min, y_max), (x_max, y_max)]
            chunks.append(chunk_corners)
                
    # 'divide and conquer approach' - 
    # first remove chunks that are fully in sensor's range
    # for sensor:
    #     if chunk in sensor range:
    #         delete chunk
    granularity = 0
    while granularity < granularity_limit:    
        print(f"\ngranularity: {granularity}")
        print(f"chunks: {len(chunks)}")                    
        for sensor in tqdm(sensor_data):
            #print(f"\nsensor: {sensor}")
            for chunk in chunks:
                if (calc_dist(chunk[0], sensor[0]) > sensor[2]) or \
                    (calc_dist(chunk[1], sensor[0]) > sensor[2]) or \
                    (calc_dist(chunk[2], sensor[0]) > sensor[2]) or \
                    (calc_dist(chunk[3], sensor[0]) > sensor[2]):
                    pass
                else:
                    chunks.remove(chunk)
                    #print(f"deleted chunk {chunk}. {len(chunks)} remain")
        print(f"chunks remaining: {len(chunks)}")
        granularity += 1
        if granularity < granularity_limit:
            chunks = split_chunk(chunks, search_size)
    
    print(f"\n\nchecking for coverage")

    for chunk in tqdm(chunks):
        #print(chunk)        
        if not is_chunk_covered(sensor_data, chunk, search_size):
            print(chunk) 
    return 0
     

input_data = read_input("inputs/day15.txt")
part2(input_data, 4000000, 2**17, 13);
#part2(input_data, 20, 2**4, 5);

number of chunks: 31*31 = 961

granularity: 0
chunks: 961


100%|███████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 2507.92it/s]


chunks remaining: 330
splitting 330 chunks
chunk_size: 131072
new chunk size: 65536
split into 1320 chunks.

granularity: 1
chunks: 1320


100%|███████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 1184.25it/s]


chunks remaining: 203
splitting 203 chunks
chunk_size: 65536
new chunk size: 32768
split into 811 chunks.

granularity: 2
chunks: 811


...

100%|█████████████████████████████████████████████████████████████████████████████| 19/19 [00:58<00:00,  3.09s/it]


chunks remaining: 36459


checking for coverage


 78%|██████████████████████████████████████████████████████▊               | 28524/36459 [01:56<00:32, 246.51it/s]

x=3292963, y=3019123
[(3292960, 3019104), (3292991, 3019104), (3292960, 3019135), (3292991, 3019135)]


100%|██████████████████████████████████████████████████████████████████████| 36459/36459 [02:29<00:00, 243.83it/s]


In [1]:
(3292963*4000000) + 3019123


13171855019123